In [1]:
import os
import tensorflow as tf
import numpy as np

In [2]:
DATASET_PATH = "Vehicle"
images = []
classes =[]
class_list = []

In [3]:
for i, classname in enumerate(os.listdir(DATASET_PATH)):
    class_list.append(classname)
    for j, filename in enumerate(os.listdir(os.path.join(DATASET_PATH, classname))):
        image = tf.keras.utils.load_img(os.path.join(DATASET_PATH, classname, filename), color_mode = "grayscale")
        image = tf.keras.utils.img_to_array(image)
        image /= 255
        images.append(image)
        classes.append(i)

In [5]:
images_resized = [tf.image.resize(img, (255, 255)).numpy() for img in images] # Idk kok ada error disini, ini fix nya
images = np.array(images_resized)
classes = np.array(classes)

In [7]:
y = tf.keras.utils.to_categorical(classes, num_classes = len(class_list))
y.shape

(3000, 3)

# Model

In [8]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [9]:
x_train, x_temp, y_train, y_temp = train_test_split(images, y, train_size=0.8)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size=0.5)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), strides = (1, 1), padding = "same", activation = "relu",  input_shape = (x_train[0].shape)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32, kernel_size = (5, 5), strides = (2, 2), padding = "same", activation = "relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate = 0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = "relu"),
    tf.keras.layers.Dropout(rate = 0.2),
    tf.keras.layers.Dense(len(class_list), activation = "softmax")
])

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics=['accuracy'])
model.summary()

d:\Apps\Conda\envs\TensorFlow\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 255, 255, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,097,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,149,283 (8.20 MB)

 Trainable params: 2,149,283 (8.20 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
hist = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 10, batch_size = 50)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 52s 981ms/step - accuracy: 0.8758 - loss: 0.3637 - val_accuracy: 0.8867 - val_loss: 0.3438
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 44s 912ms/step - accuracy: 0.9168 - loss: 0.2239 - val_accuracy: 0.8867 - val_loss: 0.3260
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 42s 879ms/step - accuracy: 0.9211 - loss: 0.2221 - val_accuracy: 0.8800 - val_loss: 0.3200
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 44s 910ms/step - accuracy: 0.9545 - loss: 0.1448 - val_accuracy: 0.8967 - val_loss: 0.3108
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 42s 866ms/step - accuracy: 0.9650 - loss: 0.1099 - val_accuracy: 0.8967 - val_loss: 0.3462
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 44s 920ms/step - accuracy: 0.9683 - loss: 0.0940 - val_accuracy: 0.8967 - val_loss: 0.3412
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 42s 885ms/step - accuracy: 0.9881 - loss: 0.0478 - val_accuracy: 0.9167 - val_loss: 0.3220
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 42s 871ms/step - accuracy: 0.9632 - loss: 0.1104 - val_accu

In [13]:
for i in range(20):
    img = np.expand_dims(x_test[i], axis = 0)
    pred = model.predict()
    
    print(f"Sample {i}: Actual Label = {class_list[np.argmax(y_test[i])]}, Predicted Label = {class_list[np.argmax(pred)]}")

TypeError: TensorFlowTrainer.predict() missing 1 required positional argument: 'x'

In [ ]:
plt.subplot(1,2,1)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.xlabel('Epochs')
plt.xlabel('Accuracy')
plt.legend(['Accuracy', "Val_Accuracy"])

plt.subplot(1,2,2)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.xlabel('Epochs')
plt.xlabel('Loss')
plt.legend(['Loss', "Val_Loss"])

plt.show()